In [35]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(torch.cuda.is_available())

True


In [36]:
# Libraries
import gc
import pandas as pd
import numpy as np
import random
import shutil
import csv
import os
import re
import time
import glob
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from os import listdir
from os.path import isfile,join
from sklearn.preprocessing import normalize
from tqdm.notebook import tqdm
from IPython.display import clear_output


In [45]:
STARTCSV_PATH = '/root/workspace/FAISS-Imputation/data-imputation/Data/files-to-process/SHUFFLED'
STARTTXT= 'iTuneAmazonS'
STARTTXTFP=STARTCSV_PATH+STARTTXT+'.txt'
STARTCSVFP=STARTTXTFP[:STARTTXTFP.rfind('.')]+'.csv'
SENTENCETRANSFORMER='sentence-transformers/LaBSE'
cumulative_percentages = [0.0, 0.1, 0.1, 0.1, 0.1]
cumulative_percentages = [sum(cumulative_percentages[:i + 1]) for i in range(len(cumulative_percentages))]
# seeds = random.sample(range(1, 10000), 10)
seeds = [98, 409, 1753, 8424, 2782, 6004, 3581, 5553, 4721, 7179]


In [47]:
# Function to process individual file content
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        giantlist = []
        for line in file:
            col_val_pairs = re.findall(r'COL(.*?)VAL(.*?)(?=COL|$)', line)
            if col_val_pairs:
                listrowi = []
                for col, val in col_val_pairs:
                    col_value = col.strip()
                    val_value = val.strip()
                    listrowi.append([col_value, val_value])
                last_val = listrowi[-1][1]
                label = str(last_val)[-1]
                listrowi[-1][1] = last_val[:-2].strip()
                listrowi.append(['label', label])
                giantlist.append(listrowi)
            else:
                print(f"No 'COL' or 'VAL' pairs found in line: {line.strip()}")
    return giantlist

# Function to save content into CSV
def save_to_csv(r, csv_file_path):
    headers = [str(x[0]) for x in r[0]]
    print(headers)
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(headers)

        for record in r:
            row = [str(x[1]) for x in record]
            csvwriter.writerow(row)

# Function to join all related files and process them
def join_and_process_files(folder_path):
    all_files = os.listdir(folder_path)
    data_files = [file for file in all_files if file.rfind(STARTTXT) != -1 and file.endswith('.txt')]

    for file_name in data_files:
        STARTTXTFP = os.path.join(folder_path, file_name)
        STARTCSVFP = STARTTXTFP[:STARTTXTFP.rfind('.')] + '.csv'
        # print(f"Processing File: {STARTTXTFP}")
        # print(f"Saving as CSV: {STARTCSVFP}")
        records = process_file(STARTTXTFP)
        save_to_csv(records, STARTCSVFP)

# Example usage
join_and_process_files(STARTCSV_PATH)

['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released', 'Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released', 'label']
['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released', 'Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released', 'label']
['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released', 'Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released', 'label']


In [ ]:

# Function to split the CSV file into 3 subsets
def split_csv(file_path):
    df = pd.read_csv(file_path, header=None, encoding='utf-8')
    df.columns = df.iloc[0]
    df = df.reindex(df.index.drop(0)).reset_index(drop=True)
    headers = list(df.columns)
    g = int((len(headers) - 1) / 2)
    subset1 = df.iloc[:, :g]
    subset2 = df.iloc[:, g:-1]
    subset3 = df.iloc[:, -1]

    # Construct paths for the subsets
    subset1_path = file_path[:file_path.rfind('.')] + '_1.csv'
    subset2_path = file_path[:file_path.rfind('.')] + '_2.csv'
    subset3_path = file_path[:file_path.rfind('.')] + '_3.csv'
    subset1.to_csv(subset1_path, index=False, header=True, encoding='utf-8')
    subset2.to_csv(subset2_path, index=False, header=True, encoding='utf-8')
    subset3 = subset3.astype(int)
    subset3.to_csv(subset3_path, index=False, header=True, encoding='utf-8')
    return subset1_path, subset2_path, subset3_path

def rejoin_splits_and_save(STARTCSV_PATH):
    all_files = os.listdir(STARTCSV_PATH)
    prefixes = set([file[:file.rfind('_')] for file in all_files if file.endswith('.csv')])

    for prefix in prefixes:
        subset1_path = os.path.join(STARTCSV_PATH, prefix + '_1.csv')
        subset2_path = os.path.join(STARTCSV_PATH, prefix + '_2.csv')

        if os.path.exists(subset1_path) and os.path.exists(subset2_path):
            subset1_loaded = pd.read_csv(subset1_path, encoding='utf-8')
            subset2_loaded = pd.read_csv(subset2_path, encoding='utf-8')

            result = pd.concat([subset1_loaded, subset2_loaded], axis=0, ignore_index=True)
            result_path = os.path.join(STARTCSV_PATH, prefix + '_12.csv')
            result.to_csv(result_path, index=False, header=True, encoding='utf-8')

            print(f"Rejoined subsets saved to {result_path}")
        else:
            print(f"Skipping {prefix} as one or both subset files are missing")
all_csv_files = [file for file in os.listdir(STARTCSV_PATH) if file.endswith('.csv') and '_12.csv' not in file]

for file in all_csv_files:
    file_path = os.path.join(STARTCSV_PATH, file)
    split_csv(file_path)
rejoin_splits_and_save(STARTCSV_PATH)
dataframes = []
for file_name in os.listdir(STARTCSV_PATH):
    if file_name.endswith('_12.csv') and file_name.startswith('test'):
        file_path = os.path.join(STARTCSV_PATH, file_name)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, encoding='utf-8')
            df['source_file'] = file_name
            dataframes.append(df)
        else:
            print(f"{file_name} does not exist.")
result = pd.concat(dataframes, axis=0, ignore_index=False)
result_path = os.path.join(STARTCSV_PATH, 'combined_all_files.csv')
result.to_csv(result_path, index=False, header=True, encoding='utf-8')

print(f"Combined CSV saved to {result_path}")
specific_files_to_remove = [
    os.path.join(STARTCSV_PATH, f'test_{STARTTXT}.csv'),
    os.path.join(STARTCSV_PATH, f'test_{STARTTXT}_1.csv'),
    os.path.join(STARTCSV_PATH, f'test_{STARTTXT}_2.csv'),
    os.path.join(STARTCSV_PATH, f'test_{STARTTXT}_12.csv'),
    os.path.join(STARTCSV_PATH, f'train_{STARTTXT}.csv'),
    os.path.join(STARTCSV_PATH, f'train_{STARTTXT}_1.csv'),
    os.path.join(STARTCSV_PATH, f'train_{STARTTXT}_2.csv'),
    os.path.join(STARTCSV_PATH, f'train_{STARTTXT}_12.csv'),
    os.path.join(STARTCSV_PATH, f'valid_{STARTTXT}.csv'),    
    os.path.join(STARTCSV_PATH, f'valid_{STARTTXT}_1.csv'),
    os.path.join(STARTCSV_PATH, f'valid_{STARTTXT}_2.csv'),
    os.path.join(STARTCSV_PATH, f'valid_{STARTTXT}_12.csv'),
]
for file_name in specific_files_to_remove:
    print(file_name)
    if os.path.isfile(file_name):
        os.remove(file_name)  

print("Cleanup complete. Only the specified files are kept.")

In [49]:
# Function to inject null values without nulling entire rows
def inject_nulls(dataframe, target_percentage, current_percentage, seed):
    np.random.seed(seed)
    random.seed(seed)
    additional_percentage = target_percentage - current_percentage
    print(f"Additional percentage needed: {additional_percentage}")

    num_rows, num_cols = dataframe.shape
    num_cols_to_inject = num_cols - 1
    null_counts = [int((num_rows - 1) * additional_percentage) for _ in range(num_cols_to_inject)]

    # Create a copy of the DataFrame to inject nulls
    modified_df = dataframe.copy()
    for col_idx in range(num_cols_to_inject):
        col_null_count = null_counts[col_idx]
        if col_null_count > 0:
            available_indices = modified_df.index[modified_df.iloc[:, col_idx].notna()].tolist()
            if len(available_indices) < col_null_count:
                col_null_count = len(available_indices)
            if col_null_count > 0 and available_indices:
                null_indices = random.sample(available_indices, col_null_count)
                modified_df.iloc[null_indices, col_idx] = np.nan

    return modified_df


# Calculate and print percentage of missing values in each column
def print_missing_value_percentage(dataframe):
    missing_percentage = dataframe.isnull().mean() * 100
    print("Percentage of missing values in each column:")
    print(missing_percentage)
    
df = pd.read_csv(result_path, encoding='utf-8')
for seed in seeds:
    print(f"Processing seed: {seed}")
    current_df = df.copy()
    current_percentage = 0
    
    for target_percentage in cumulative_percentages:
        print(f"Processing percentage: {target_percentage} for seed {seed}")
        modified_df = inject_nulls(current_df, target_percentage, current_percentage, seed)
        current_df = modified_df
        current_percentage = target_percentage

        print(f"\nMissing values for {int(target_percentage * 100)}% nulls (seed {seed}):")
        print_missing_value_percentage(modified_df)

        output_file_name = os.path.join(STARTCSV_PATH, f'{STARTTXT}_{(current_percentage)}_{seed}_nonimputed.csv')
        modified_df.to_csv(output_file_name, index=False, header=True, encoding='utf-8')
        print(f"Saved: {output_file_name}")

In [51]:
# Imputation using FAISS

def get_map_dq_d1n(name):
    if "DBLPACM" in name:
        return [0,1,2,3]
    if "walmartAmazon" in name:
        return [0,1,2,3,4]
    if "DBLPScholar" in name:
        return [0,1,2,3]

def impute_values(dq, dfaiss, map_dq_d1n, similarity_threshold):
    results = []
    for row in dq.values:
        query = ' '.join([str(i) if not pd.isna(i) else " " for i in row[:-1]])
        docs_and_scores = dfaiss.similarity_search_with_relevance_scores(query)
        
        row_copy = row.copy().tolist()

        for i in range(len(row)):
            if pd.isna(row_copy[i]):
                missing_position = i
                for emb, score in docs_and_scores:
                    if score >= similarity_threshold:
                        candidate = emb.metadata[feature_names_d1n[map_dq_d1n[missing_position]]]
                        row_copy[missing_position] = candidate
                        if not pd.isna(row_copy[missing_position]):
                            break
        results.append(row_copy)

    results = pd.DataFrame(results, columns=feature_names_d1n)
    return results

# MAIN IMPUTATION LOOP
for seed in seeds:
    for null_percentage in ["0.0", "0.1", "0.2", "0.3", "0.4"]:
        initail_dataset = STARTCSV_PATH + "/" + STARTTXT + "_" + str(null_percentage) + "_" + str(seed) + "_nonimputed.csv"
        cs = pd.read_csv(initail_dataset, encoding='utf-8')
        d1n = cs
        feature_names_d1n = list(d1n.columns.values)
        map_dq_d1n = get_map_dq_d1n(STARTTXT)
        docs = []

        for row in d1n.values:
            key = ' '.join([str(i) if not pd.isna(i) else " " for i in row[:-1]])
            docs.append(Document(page_content=key, metadata={k: v for k, v in zip(feature_names_d1n, row)}))

        embedding_model = HuggingFaceEmbeddings(model_name=SENTENCETRANSFORMER)
        dfaiss = FAISS.from_documents(docs, embedding_model)

        initail_dataset_imputed_fp = STARTCSV_PATH + "/" + STARTTXT + "_" + str(null_percentage) + "_" + str(seed) + "_imputed.csv"
        dq2 = cs
        dq2.isnull().mean() * 100
        feature_names_dq2 = list(dq2.columns.values)
        results2 = impute_values(dq2, dfaiss, map_dq_d1n, 0.50)
        results2.to_csv(initail_dataset_imputed_fp, index=False)

        del dfaiss
        gc.collect()
        print("FAISS file removed for " + initail_dataset)


# Clear the output of the executed cell
clear_output(wait=True)
print("The Process is Completed Successfully!")


The Process is Completed Successfully!


In [52]:
# Function to split dataframe based on the last column (source_file)
def split_based_on_last_column(dataframe, percentage, seed, imputed_status, STARTCSV_PATH, original_file):
    unique_labels = dataframe.iloc[:, -1].unique()
    print(f"Unique labels found: {unique_labels}")

    label_mapping = {
        'train': 'training_file',
        'test': 'testing_file',
        'valid': 'validation_file'
    }

    files_created = []
    for label in unique_labels:
        label_core = label.replace('.csv', '').split('_')[0].lower()
        new_label = label_mapping.get(label_core, None)

        if new_label is None:
            print(f"Skipping unknown label: {label}")
            continue
        subset_df = dataframe[dataframe.iloc[:, -1] == label]
        print(f"Processing label: {new_label}, Rows: {len(subset_df)}")

        # Construct output file path
        output_file_name = os.path.join(STARTCSV_PATH, f'{new_label}_{percentage}_{seed}_{imputed_status}.csv')

        # Save the subset to a new CSV file
        if not subset_df.empty:
            subset_df.to_csv(output_file_name, index=False, header=True, encoding='utf-8')
            print(f"Saved: {output_file_name}")
            remove_last_column(output_file_name)
            files_created.append(output_file_name)
        else:
            print(f"No data to save for {new_label}.")
def remove_last_column(file_path):
    try:
        # Read the CSV
        df = pd.read_csv(file_path, encoding='utf-8')
        df = df.iloc[:, :-1]
        df.to_csv(file_path, index=False, header=True, encoding='utf-8')
        print(f"Removed last column and saved: {file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
# Process each file in the folder
for file_name in os.listdir(STARTCSV_PATH):
    if file_name.startswith(f'{STARTTXT}') and (file_name.endswith('_imputed.csv') or file_name.endswith('_nonimputed.csv')):
        print(f"Processing file: {file_name}")
        parts = file_name.replace('.csv', '').split('_')
        if len(parts) == 4:
            percentage = parts[1]
            seed = parts[2]
            imputed_status = parts[3]

            file_path = os.path.join(STARTCSV_PATH, file_name)

            try:
                df = pd.read_csv(file_path, encoding='utf-8')
            except Exception as e:
                print(f"Error reading {file_name}: {e}")
                continue

            if df.empty:
                print(f"DataFrame is empty for file: {file_name}")
                continue

            # Split the dataframe based on the last column and delete source file
            split_based_on_last_column(df, percentage, seed, imputed_status, STARTCSV_PATH, file_path)
        else:
            print(f"File name format not matching expected pattern: {file_name}")
    else:
        print(f"Skipping file: {file_name} (doesn't match)")

file_patterns = ["testing_file_*_*_imputed.csv", "training_file_*_*_imputed.csv", "validation_file_*_*_imputed.csv", 
                 "testing_file_*_*_nonimputed.csv", "training_file_*_*_nonimputed.csv", "validation_file_*_*_nonimputed.csv"]
additional_files = {
    "testing_file": f'test_{STARTTXT}_3.csv',
    "training_file": f'train_{STARTTXT}_3.csv',
    "validation_file": f'valid_{STARTTXT}_3.csv'
}

def split_file_horizontally(file_path, save_dir):
    df = pd.read_csv(file_path)
    if len(df) < 2:
        print(f"Error: {file_path} does not have enough rows to split.")
        return None, None
    half_index = len(df) // 2
    df_first_half = df.iloc[:half_index, :]
    df_second_half = df.iloc[half_index:, :]
    base_name = os.path.basename(file_path).replace('.csv', '')
    first_half_file = os.path.join(save_dir, f"{base_name}_first_half.csv")
    second_half_file = os.path.join(save_dir, f"{base_name}_second_half.csv")

    df_first_half.to_csv(first_half_file, index=False)
    df_second_half.to_csv(second_half_file, index=False)

    print(f"Split {file_path} into:")
    print(f"  - {first_half_file} with shape {df_first_half.shape}")
    print(f"  - {second_half_file} with shape {df_second_half.shape}")

    return first_half_file, second_half_file
def merge_files_horizontally(file1_path, file2_path, save_dir):
    df_first_half = pd.read_csv(file1_path)
    df_second_half = pd.read_csv(file2_path)
    df_merged = pd.concat([df_first_half, df_second_half], axis=1)

    # Save the merged DataFrame
    base_name = os.path.basename(file1_path).replace('_first_half.csv', '')
    merged_file = os.path.join(save_dir, f"{base_name}_merged.csv")
    df_merged.to_csv(merged_file, index=False)

    print(f"Merged {file1_path} and {file2_path} into:")
    print(f"  - {merged_file} with shape {df_merged.shape}")

    return merged_file

# Function to merge the additional file based on naming pattern
def merge_additional_file(merged_file_path, additional_file_name, save_dir):
    df_merged = pd.read_csv(merged_file_path)
    df_additional = pd.read_csv(os.path.join(save_dir, additional_file_name))
    df_final = pd.concat([df_merged, df_additional], axis=1)
    final_file_name = merged_file_path.replace('_merged.csv', '_final.csv')
    df_final.to_csv(final_file_name, index=False)

    print(f"Merged {merged_file_path} with {additional_file_name} into:")
    print(f"  - {final_file_name} with shape {df_final.shape}")

    return merged_file_path, final_file_name
files_to_delete = []

# Loop through each pattern and process matching files
for pattern in file_patterns:
    files = glob.glob(os.path.join(STARTCSV_PATH, pattern))

    for file_path in files:
        print(f"Processing file: {file_path}")
        first_half_file, second_half_file = split_file_horizontally(file_path, STARTCSV_PATH)
        if first_half_file and second_half_file:
            merged_file = merge_files_horizontally(first_half_file, second_half_file, STARTCSV_PATH)

            # Determine the additional file to merge based on the naming pattern
            for key, additional_file in additional_files.items():
                if key in pattern:
                    final_file_name = merge_additional_file(merged_file, additional_file, STARTCSV_PATH)
                    files_to_delete.extend([first_half_file, second_half_file, merged_file])

for file in files_to_delete:
    os.remove(file)
    print(f"Deleted file: {file}")
files_to_remove_patterns = [
    os.path.join(STARTCSV_PATH, 'testing_file_*_*_imputed.csv'),
    os.path.join(STARTCSV_PATH, 'testing_file_*_*_nonimputed.csv'),
    os.path.join(STARTCSV_PATH, 'training_file_*_*_imputed.csv'),
    os.path.join(STARTCSV_PATH, 'training_file_*_*_nonimputed.csv'),
    os.path.join(STARTCSV_PATH, 'validation_file_*_*_imputed.csv'),
    os.path.join(STARTCSV_PATH, 'validation_file_*_*_nonimputed.csv'),
    os.path.join(STARTCSV_PATH, f'test_{STARTTXT}_3.csv'),
    os.path.join(STARTCSV_PATH, f'train_{STARTTXT}_3.csv'),
    os.path.join(STARTCSV_PATH, f'valid_{STARTTXT}_3.csv'),
    
]

for pattern in files_to_remove_patterns:
    files_to_remove = glob.glob(pattern)
    for file_path in files_to_remove:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted file: {file_path}")
        else:
            print(f"File not found or not a file: {file_path}")

# Function to convert a CSV file to a TXT file
def write_to_text(csv_file_path):
    text_file_path = csv_file_path[:csv_file_path.rfind('.')] + ".txt"
    data = []

    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        headers = next(csvreader)

        for row in csvreader:
            record = []
            for i in range(len(headers)):
                if row[i].strip():
                    record.append([headers[i].strip(), row[i].strip()])
                else:
                    record.append([headers[i].strip(), ''])
            data.append(record)

        with open(text_file_path, 'w', encoding='utf-8') as file:
            names = headers
            for record in data:
                next_index = 0
                last_value_written = ""

                for i, pair in enumerate(record):
                    col, val = pair
                    while next_index < len(names) and col != names[next_index]:
                        if next_index == ((len(names) - 1) // 2 - 1):
                            file.write(f"COL {names[next_index]} VAL  \t")
                        elif names[next_index] == 'label':
                            pass
                        else:
                            file.write(f"COL {names[next_index]} VAL ")
                        next_index += 1
                    if next_index < len(names):
                        if next_index == ((len(names) - 1) // 2 - 1):
                            file.write(f"COL {col} VAL {val} \t")
                        elif col == 'label':
                            last_value_written = val
                        else:
                            file.write(f"COL {col} VAL {val} ")
                        next_index += 1
                while next_index < len(names):
                    if next_index == ((len(names) - 1) // 2 - 1):
                        file.write(f"COL {names[next_index]} VAL \t")
                    elif names[next_index] == 'label':
                        pass
                    else:
                        file.write(f"COL {names[next_index]} VAL ")
                    next_index += 1
                file.write(f"\t{last_value_written}\n")


# Get all final CSV files in the directory (change the pattern if needed)
final_csv_files = glob.glob(os.path.join(STARTCSV_PATH, "*_final.csv"))

for csv_file in final_csv_files:
    print(f"Converting {csv_file} to TXT...")
    write_to_text(csv_file)

print("Conversion completed.")
file_patterns = [
    "testing_file_*_*_imputed_final.csv",
    "training_file_*_*_imputed_final.csv",
    "validation_file_*_*_imputed_final.csv",
    "testing_file_*_*_nonimputed_final.csv",
    "training_file_*_*_nonimputed_final.csv",
    "validation_file_*_*_nonimputed_final.csv"
]
def delete_files_by_pattern(base_path, file_patterns):
    for pattern in file_patterns:
        files = glob.glob(os.path.join(base_path, pattern))
        print(f"Found files matching pattern '{pattern}': {files}")
        for file_path in files:
            try:
                os.remove(file_path)
                print(f"Deleted file: {file_path}")
            except Exception as e:
                print(f"Error deleting file {file_path}: {e}")
delete_files_by_pattern(STARTCSV_PATH, file_patterns)

from IPython.display import clear_output

# Clear the output of the executed cell
clear_output(wait=True)
print("The Process is Completed Successfully!")

The Process is Completed Successfully!


In [53]:
folder_structure = {
    f'imputed_{STARTTXT}': '*_imputed_final.txt',
    f'non-imputed_{STARTTXT}': '*_nonimputed_final.txt',
    f'imputed_CSV_{STARTTXT}': '*.csv'
}

TARGET_PATH_1 = '/root/workspace/FAISS-Imputation/data-imputation/Data/processed_files/iTunes-Amazon/THRESHOLD50'
TARGET_PATH_2 = '/root/workspace/FAISS-Imputation/ditto-master/data/SHUFFLED/iTunes-Amazon/THRESHOLD50'
def create_main_folders(target_paths):
    for target_path in target_paths:
        os.makedirs(target_path, exist_ok=True)
        print(f"Created main folder: {target_path}")
def create_folders(target_paths, folder_structure):
    for target_path in target_paths:
        for subfolder in folder_structure:
            subfolder_path = os.path.join(target_path, subfolder)
            os.makedirs(subfolder_path, exist_ok=True)
            print(f"Created folder: {subfolder_path}")
def copy_and_remove_files(target_paths, folder_structure):
    for subfolder, pattern in folder_structure.items():
        files = glob.glob(os.path.join(STARTCSV_PATH, pattern))
        print(f"Looking for files with pattern: {pattern}")
        print(f"Found files: {files}")
        for file_path in files:
            for target_path in target_paths:
                target_folder = os.path.join(target_path, subfolder)
                shutil.copy(file_path, target_folder)
                print(f"Copied {file_path} to {target_folder}")
            os.remove(file_path)
            print(f"Removed {file_path} from source directory")
def delete_folder_from_target(target_path, folder_name):
    folder_path = os.path.join(target_path, folder_name)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    else:
        print(f"Folder {folder_path} does not exist.")
target_paths = [TARGET_PATH_1, TARGET_PATH_2]
create_main_folders(target_paths)

# Create the subfolders in both target directories
create_folders(target_paths, folder_structure)
copy_and_remove_files(target_paths, folder_structure)
delete_folder_from_target(TARGET_PATH_2, f'imputed_CSV_{STARTTXT}')


The Process is Completed Successfully!
